In [3]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

client = MongoClient(os.getenv("MONGO_DB_CONNECTION_STRING"))
db = client["vms"]
 
invitesCollection = db["invites"]
groupInvitesCollection = db["groupinvites"]
groupParkingReservationsCollection = db["groupParkingReservations"]
parkingReservationCollection = db["parkingreservations"]
trayCollection = db["trays"]
test = db["testgroup"]
# test.delete_many({})
# groupInvitesCollection.delete_many({})
# groupParkingReservationsCollection.delete_many({})


# print(invitesCollection.find_one({})['inviteDate'])
# numVis = invitesCollection.count_documents({ '$and': [ { "inviteState": { '$ne': "inActive" } }, { "inviteState": { '$ne': "cancelled" } } ] })
# print(numVis)

In [9]:
print(list(invitesCollection.aggregate([
    {
        '$group': {
            '_id': '$visitorEmail', 
            'visitorName': {
                '$first': '$visitorName'
            }, 
            'numInvites': {
            '$count': {}
            },
            'visits': 
            {
                '$push': {
                    'date': '$inviteDate'
                }
            }
        }
    }
])))

OperationFailure: The field 'visitorName' must be an accumulator object, full error: {'ok': 0.0, 'errmsg': "The field 'visitorName' must be an accumulator object", 'code': 40234, 'codeName': 'Location40234', '$clusterTime': {'clusterTime': Timestamp(1661265221, 2), 'signature': {'hash': b' \x91\xcf\x85d\x8f5^f\xd2\xc9\xc5\\\xfa\x82$~*\xc7u', 'keyId': 7086437743361785861}}, 'operationTime': Timestamp(1661265221, 2)}

In [13]:
parkingReservationCollection.aggregate([
    {
        '$group': {
            '_id': '$reservationDate', 
            'numParkings': {
                '$sum': 1
            }
        }
    }, {
        '$out': 'groupparkingreservations'
    }
    ])

In [56]:
# db.collection.aggregate([
#     { "$group": {
#         "_id": { "checkType": "$_id.checkType", "resultCode": "$_id.resultCode" },
#         "count": { "$sum": "$count" },
#         "ctv": { "$sum": "$ctv" },
#         "details": { "$push": "$$ROOT" }
#     } },
#     { "$project": {
#         "count": 1,
#         "details": 1,
#         "ctv": 1,
#         "perc": { "$multiply": [ { "$divide": ["$ctv","$count"] }, 100 ] },
#         "weight": { "$divide": ["$ctv", "$count"] },
#     } }
# ])
test.delete_many({})
invitesCollection.aggregate([
    {
        '$group': {
            '_id': '$inviteDate', 
            'numInvites': {
                '$sum': 1
            },
            'numVisitors':{
                '$sum': {
                        '$add': [
                            {'$cond': [
                                {
                                    '$and' : [ 
                                        {
                                            '$ne': [
                                                '$inviteState', 'inActive'
                                            ]
                                        },
                                        {
                                            '$ne': [
                                                '$inviteState', 'cancelled'
                                            ]
                                        }
                                    ]
                                }, 1, 0
                            ]},{
                                '$cond': [
                                    {
                                        '$and': [
                                            {
                                                '$eq': [
                                                    '$inviteState', 'extended'
                                                ]
                                            }, 
                                            {
                                                '$gte': [
                                                    '$signInTime', '$inviteDate'
                                                ]
                                            },
                                            {
                                                '$lte': [
                                                    '$signOutTime', '$inviteDate'
                                                ]
                                            }
                                        ]
                                    }, 
                                    { 
                                        '$sum': 1   
                                    },0
                                ]
                            }
                        ]
                    }
                }
            }
    },
    {
        '$out': 'testgroup'
    }
    ])

OperationFailure: Unrecognized expression '$count', full error: {'ok': 0.0, 'errmsg': "Unrecognized expression '$count'", 'code': 168, 'codeName': 'InvalidPipelineOperator', '$clusterTime': {'clusterTime': Timestamp(1661258233, 1), 'signature': {'hash': b'\x0b\xfa\xbb\x12\xab9\x07\xa5j`\x86=w\x87]>\xb5\x8e\xbaq', 'keyId': 7086437743361785861}}, 'operationTime': Timestamp(1661258233, 1)}

In [51]:
# invitesCollection.insert_one({
#       "inviteID": "lar4f869-22c1-11ed-a4bo-3468959t8h96",
#       "userEmail": "larisabotha@icloud.vom",
#       "visitorEmail": "larisabotha@gmail.com",
#       "idDocType": "RSA-ID",
#       "idNumber": "0012120178087",
#       "inviteDate": "2022-08-23",
#       "inviteState": "extended",
#       "visitorName": "Larisa",
#       "relation": "family",
#       "requiresParking": False,
#       "notes": "So kind",
#       "signInTime": "2022-08-23 10:38:12",
#       "signOutTime": "2022-08-29 11:27:22"
#   })

invitesCollection.insert_one({
    "inviteID": "law4f769-21c1-11ed-a4bo-3468959t6h96",
    "userEmail": "larisabotha@icloud.vom",
    "visitorEmail": "larisabotha@gmail.com",
    "idDocType": "RSA-ID",
    "idNumber": "0012120178087",
    "inviteDate": "2022-08-23",
    "inviteState": "signedIn",
    "visitorName": "Larisa",
    "relation": "family",
    "requiresParking": False,
    "notes": "So kind",
    "signInTime": "2022-08-23 10:38:12",
    "signOutTime": "2022-08-23 11:27:22"
})

In [17]:
invitesCollection.aggregate([
    {
        '$group': {
            '_id': '$inviteDate', 
            'numInvites': {
                '$sum': 1
            }, 
            'numVisitors': {
                '$add': [{
                    '$cond': [
                        {
                            '$and' : [ 
                                {'$ne': [
                                '$inviteState', 'inActive'
                                ]},
                                {'$ne': [
                                '$inviteState', 'cancelled'
                                ]}
                            ]
                        }, 1, 0
                    ]
                }, {
                  '$cond': [
                        {
                                '$eq': [
                                '$inviteState', 'extended'
                                ]
                        }, { '$subtract' : [ '$signOutTime', "$signInTime" ] }, 0
                    ]
                }]
            }
        }
    }, {
        '$out': 'groupinvites'
    }
    ])

OperationFailure: The $sum accumulator is a unary operator, full error: {'ok': 0.0, 'errmsg': 'The $sum accumulator is a unary operator', 'code': 40237, 'codeName': 'Location40237', '$clusterTime': {'clusterTime': Timestamp(1661243139, 2), 'signature': {'hash': b'\xfa]@\xa1/6Z\xda\x89-\xbc\x15\xc3\xd2\x1b\xd0\xb6F\xed[', 'keyId': 7086437743361785861}}, 'operationTime': Timestamp(1661243139, 2)}

In [18]:
import json
import numpy as np
l = list(groupInvitesCollection.find({},{"_id":0 ,"numInvites": 1}))

print(np.min(l))

TypeError: the JSON object must be str, bytes or bytearray, not list